In [21]:
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt 
import pandas as pd 
#import backtesting as bt
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas_ta as ta
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

def BOLL(values, n, m):
    """
    Return Bollinger Bands for `values`, at
    each step taking into account `n` previous values
    and `m` standard deviations away from the mean.
    """
    sma = SMA(values, n)
    std = pd.Series(values).rolling(n).std()
    return sma, sma + m * std, sma - m * std

def Dochian(data, high_length, low_length):
    """
    Return Dochian Channel for `values`, at
    each step taking into account `n` previous values
    """
    return ta.donchian(data.High.s, data.Low.s, low_length, high_length, offset=1, talib=False)

def ATR(data, n):
    return np.array(ta.atr(high=data.High.s, low=data.Low.s, close=data.Close.s, length=n, talib=False))
    
    # tr = pd.Series(np.maximum(high - low, high - close, close - low))
    # return tr.rolling(n).mean()
class turtleTrading(Strategy):
    # Define the two MA lags as *class variables*
    
    def init(self):
        print(ATR(self.data, 20))
        self.atr = self.I(ATR, self.data, 20)
        self.donchian = self.I(Dochian, self.data, 20, 10)
        self.last_buy_price = 0
        self.last_buy_atr = 0
        
    def next(self):
        
        # If today's close exceed the donchian_upper_bound, buy, record price and ATR
        if crossover(self.data.Close, self.donchian[2]):
            portion = (0.01*self.data.Close[-1])/self.atr[-1]
            self.buy(size=portion)
            self.last_buy_price = self.data.Close[-1]
            self.last_buy_atr = self.atr[-1]
        # If today's close is below the donchian_lower_bound, sell (end trade)
        elif crossover(self.donchian[0], self.data.Close):
            self.position.close() 
        # If price decresed by twice N at the last buy point, sell all (stop loss)
        elif crossover(self.last_buy_price - 2*self.last_buy_atr, self.data.Close[-1]):
            self.position.close()
            

bt = Backtest(GOOG, turtleTrading, cash=10_000)
stats = bt.run()
print(stats)
bt.plot()

[        nan         nan         nan ... 12.79430102 12.51508597
 12.43883167]
Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   49.534451
Equity Final [$]                     25128.84
Equity Peak [$]                      26205.67
Return [%]                           151.2884
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   11.416011
Volatility (Ann.) [%]               17.317979
Sharpe Ratio                           0.6592
Sortino Ratio                        1.179228
Calmar Ratio                         0.536468
Max. Drawdown [%]                  -21.279953
Avg. Drawdown [%]                   -4.432878
Max. Drawdown Duration     1347 days 00:00:00
Avg. Drawdown Duration       98 days 00:00:00
# Trades                                  121
Win Rate [%]                        50.413223
Best Trade [%]                      41.597398
W

/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p8158', ...)